### Sponsor / MVNO change for subscribers in DMI

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
from tqdm import tqdm
from importlib import reload
from os.path import join
import re

import roamability as rb

In [44]:
#######################################################################################
# Define variables here
#######################################################################################

downloads = r'c:\Users\balob\Documents\GITLAB\RB_BD\DATA'
out_file = 'toc_sponsor.rcjson'
out_file_mvno = 'toc_mvno.rcjson'

mvno_dict = {'Roamability':'Freecell'}
sponsor_dict = {'Partner_All':'Partner_FreeCell', 'MB_ALL': 'MB_FreeCell', 'P4_All': 'P4_FreeCell'}

# mvno_dict = {'Roamability':'TECHMOB'}
# sponsor_dict = {'P4_Combined':'P4_TOTT01', 'Partner_Combined': 'Partner_TOTT01','SMART_Tottol': 'SMART_TOTT01',
#                'MB_Tottolli': 'MB_TOTT01', 'S6_ALL': 'S6_TOTT01', 'S8_Tottolli': 'S8_TOTT01'}

# Ranges
ranges = """
454030227307000-454030227308999,
454030227310000-454030227318999,
260060149910000-260060149934999,
260060149940000-260060149949999,
260060149991909-260060149991921,
260060149997000-260060149997999,
425019613795500-425019613796499,
425019613930000-425019613944999,
425019613945001-425019613954999,
425019613960000-425019613969999,
425019613998901-425019613998913,
454030227310000-454030227318999,
"""

#######################################################################################

#######################################################################################

# toc cspadmin MoRDVruBVEpXXk6kYQ6D services1

In [45]:
# Define Class

class ImsiRange:

    def __init__(self, ranges, mvno_dict, sponsor_dict):
        self.range_list = self.parse_ranges(ranges)
        self.mvno_dict = mvno_dict
        self.sponsor_dict = sponsor_dict
    
    def parse_ranges(self, ranges):
        """Convert ranges to list"""
        pattern_clean = re.compile('([\n\s])')
        ranges = re.sub(pattern_clean, '', ranges)
        pattern_range = re.compile(r'(\d{15})\D+(\d{15})')
        range_list = []
        matches = pattern_range.finditer(ranges)
        for match in matches:
            range_list.append([match.group(1),match.group(2)])
        return range_list
    
    def prepare_range_list_for_sql(self):
        """Convert ranges in the list to string for SQL"""
        str_range_list_for_sql = ''
        for i, ranges in enumerate(imsi_range.range_list):
            str_range_list_for_sql = str_range_list_for_sql + f'(IMSI BETWEEN {ranges[0]} AND {ranges[1]})'
            if i+1 < len(self):
                str_range_list_for_sql = str_range_list_for_sql + ' OR '
        return str_range_list_for_sql
        
    def check_sponsor(self):
        sql_str = """
SELECT
SUBSTR(s.IMSI,1,12) AS IMSI_RANGE,
SUBSTR(si.IMSI_NUMBER,1,12) AS S_IMSI_RANGE,
sp.NAME AS PROFILE_NAME,
COUNT(*) AS NUM
FROM S_IMSI si, SUBSCRIBER s, SPONSOR sp WHERE
si.SUBSCRIBER_REF = s.RI
AND si.SPONSOR_REF = sp.RI
AND si.SUBSCRIBER_REF IN
(
SELECT RI FROM SUBSCRIBER WHERE
{}
)
GROUP BY SUBSTR(s.IMSI,1,12),SUBSTR(si.IMSI_NUMBER,1,12),sp.NAME
ORDER BY SUBSTR(s.IMSI,1,12)""".format(self.prepare_range_list_for_sql())
        with rb.OracleConnect('DMI', 'dd607605ce341', 'DMI') as cnxn:
            df_dmi_sponsor_gr = pd.read_sql_query(sql_str, cnxn, coerce_float=False) 
        display(df_dmi_sponsor_gr)
        return df_dmi_sponsor_gr
        
    def get_data_for_sponsor_change(self):
        sql_str = """
SELECT
si.CLI_MSISDN,s.IMSI,si.PROF_ID,si.IMSI_NUMBER,si.flag_confirmed,sp.NAME
FROM S_IMSI si, SUBSCRIBER s, SPONSOR sp WHERE
si.SUBSCRIBER_REF = s.RI
AND si.SPONSOR_REF = sp.RI
AND
({})
ORDER BY s.IMSI,si.PROF_ID""".format(self.prepare_range_list_for_sql())
        with rb.OracleConnect('DMI', 'dd607605ce341', 'DMI') as cnxn:
            df_dmi_sponsor = pd.read_sql_query(sql_str, cnxn, coerce_float=False)
        return df_dmi_sponsor
    
    def prepare_toc_to_cange_sponsor(self, downloads, out_file):
        df_dmi_imsi = self.get_data_for_sponsor_change()
        display(df_dmi_imsi.groupby('NAME',as_index=False)['IMSI'].count())
        df_dmi_imsi.NAME.replace(to_replace=self.sponsor_dict, inplace=True)
        display(df_dmi_imsi.groupby('NAME',as_index=False)['IMSI'].count())
        display(df_dmi_imsi.tail(3))
        ouf=open(join(downloads, out_file), 'w')
        t1='dmi.s_imsi.modify,CLI_MSISDN="'
        t2='",FLAG_CONFIRMED="0",IMSI_NUMBER="'
        t3='",PROF_ID="'
        t4='",SPONSOR_REF="'
        t5='",SUBSCRIBER_REF="'
        t6='";'
        for i, [cli, imsi, pid, simsi, sponsor] in enumerate(
            df_dmi_imsi[['CLI_MSISDN','IMSI','PROF_ID','IMSI_NUMBER','NAME']].values):
            ouf.write(t1+str(cli)+t2+str(simsi)+t3+str(pid)+t4+str(sponsor)+t5+str(imsi)+t6+'\n')
        ouf.close()
        
    def check_mvno(self):
        sql_str = """
SELECT
m.NAME AS MVNO_NAME,
SUBSTR(IMSI,1,12) AS IMSI_RANGE,
COUNT(*) AS NUM_SUBS,
MIN(IMSI) AS MIN_IMSI,MAX(IMSI) AS MAX_IMSI
FROM SUBSCRIBER s, MVNO m WHERE
m.RI = s.MVNO_REF
AND (
{}
)
GROUP BY m.NAME, SUBSTR(IMSI,1,12)""".format(self.prepare_range_list_for_sql())
        with rb.OracleConnect('DMI', 'dd607605ce341', 'DMI') as cnxn:
            df_dmi_mvno_gr = pd.read_sql_query(sql_str, cnxn, coerce_float=False) 
        display(df_dmi_mvno_gr)
        return df_dmi_mvno_gr
        
    def get_data_for_mvno_change(self):
        sql_str = """
SELECT
s.SIM_APP_VER,
m.NAME AS MVNO,
s.IMSI,
s.MSISDN
FROM SUBSCRIBER s, MVNO m
WHERE
s.MVNO_REF = m.RI
AND
({})""".format(self.prepare_range_list_for_sql())
        with rb.OracleConnect('DMI', 'dd607605ce341', 'DMI') as cnxn:
            df_dmi_mvno = pd.read_sql_query(sql_str, cnxn, coerce_float=False)
        return df_dmi_mvno
    
    def prepare_toc_to_cange_mvno(self, downloads, out_file):
        df_dmi_imsi = self.get_data_for_mvno_change()
        display(df_dmi_imsi.groupby('MVNO',as_index=False)['IMSI'].count())
        df_dmi_imsi.MVNO.replace(to_replace=self.mvno_dict, inplace=True)
        display(df_dmi_imsi.groupby('MVNO',as_index=False)['IMSI'].count())
        display(df_dmi_imsi.tail(3))
        ouf=open(join(downloads, out_file), 'w')
        t1='dmi.subscriber.modify,IMSI="'
        t2='",MVNO_REF="'
        t3='";'
        for i, [imsi, mvno] in enumerate(df_dmi_imsi[['IMSI','MVNO']].values):
            ouf.write(t1+str(imsi)+t2+str(mvno)+t3+'\n')
        ouf.close()
        
    def _imsis(self, df):
        for  imsi in df.IMSI:
            yield imsi

    def make_ranges(self, downloads, file_name, print_panges=False):
        df = pd.read_csv(join(downloads, file_name))
        df.sort_values(by='IMSI', inplace=True)

        imsi_num = df.count().IMSI

        ranges_list = []

        gen = self._imsis(df)
        imsi1 = next(gen)

        while True:
            range_list = [imsi1, imsi1, 1]
            try:
                while True:
                    imsi2 = next(gen)
                    if imsi2 - imsi1 == 1:
                        range_list[1] = imsi2
                        imsi1 = imsi2
                    else:
                        range_list[2] = range_list[1] - range_list[0] + 1
                        ranges_list.append(range_list)
                        imsi1 = imsi2
                        break
            except StopIteration:
                break

        range_list[2] = range_list[1] - range_list[0] + 1
        ranges_list.append(range_list)

        df = DataFrame(ranges_list, columns=['IMSI_START', 'IMSI_END', 'NUM'])

        if imsi_num == df.NUM.sum():
            print(f'OK: {imsi_num} subscribers in {len(df)} ranges')
        else:
            print(f'Wrong IMSI count: {imsi_num} in file, {df.NUM.sum()} in ranges')
            
        if print_panges:
            for range_list in df.values:
                print(f'{range_list[0]}-{range_list[1]},')
        
        return df
    
    def __len__(self):
        return len(self.range_list)
    
    def __str__(self):
        """Print ranges"""
        for i, ranges in enumerate(self.range_list):
            print(f'{i+1:02} : {ranges[0]} - {ranges[1]} -> {1+int(ranges[1])-int(ranges[0])}')
        return ''
    
imsi_range = ImsiRange(ranges, mvno_dict, sponsor_dict)

print(imsi_range)

01 : 454030227307000 - 454030227308999 -> 2000
02 : 454030227310000 - 454030227318999 -> 9000
03 : 260060149910000 - 260060149934999 -> 25000
04 : 260060149940000 - 260060149949999 -> 10000
05 : 260060149991909 - 260060149991921 -> 13
06 : 260060149997000 - 260060149997999 -> 1000
07 : 425019613795500 - 425019613796499 -> 1000
08 : 425019613930000 - 425019613944999 -> 15000
09 : 425019613945001 - 425019613954999 -> 9999
10 : 425019613960000 - 425019613969999 -> 10000
11 : 425019613998901 - 425019613998913 -> 13
12 : 454030227310000 - 454030227318999 -> 9000



In [9]:
# Prepare ranges from file with IMSIs

input_file = 'Soft_SIMs.csv'

# The input file example:

# IMSI
# 260060149999004
# 260060149999007

df_temp = imsi_range.make_ranges(downloads, input_file, print_panges=False)
df_temp.head()
display(df_temp)
for vals in df_temp.values:
    print(f'{vals[0]}-{vals[1]},')

OK: 83025 subscribers in 11 ranges


,IMSI_START,IMSI_END,NUM
0,260060149910000,260060149934999,25000
1,260060149940000,260060149949999,10000
2,260060149991909,260060149991921,13
3,260060149997000,260060149997999,1000
4,425019613795500,425019613796499,1000
5,425019613930000,425019613944999,15000
6,425019613945001,425019613954999,9999
7,425019613960000,425019613969999,10000
8,425019613998901,425019613998913,13
9,454030227307000,454030227308999,2000


260060149910000-260060149934999,
260060149940000-260060149949999,
260060149991909-260060149991921,
260060149997000-260060149997999,
425019613795500-425019613796499,
425019613930000-425019613944999,
425019613945001-425019613954999,
425019613960000-425019613969999,
425019613998901-425019613998913,
454030227307000-454030227308999,
454030227310000-454030227318999,


In [46]:
# Check MVNO in DMI

df_mvno_temp = imsi_range.check_mvno()

,MVNO_NAME,IMSI_RANGE,NUM_SUBS,MIN_IMSI,MAX_IMSI
0,Freecell,260060149943,1000,260060149943000,260060149943999
1,Freecell,454030227311,1000,454030227311000,454030227311999
2,Freecell,454030227313,1000,454030227313000,454030227313999
3,Freecell,454030227318,1000,454030227318000,454030227318999
4,Freecell,260060149997,1000,260060149997000,260060149997999
...,...,...,...,...,...
83,Freecell,425019613948,1000,425019613948000,425019613948999
84,Freecell,425019613954,1000,425019613954000,425019613954999
85,Freecell,260060149915,1000,260060149915000,260060149915999
86,Freecell,260060149918,1000,260060149918000,260060149918999


In [39]:
# Prepare TOC commands to change the MVNO in DMI

imsi_range.prepare_toc_to_cange_mvno(downloads, out_file_mvno)

,MVNO,IMSI
0,Roamability,1000


,MVNO,IMSI
0,Freecell,1000


,SIM_APP_VER,MVNO,IMSI,MSISDN
997,7,Freecell,454030227310873,8529750001873
998,7,Freecell,454030227310874,8529750001874
999,7,Freecell,454030227310875,8529750001875


In [49]:
# Check Sponsors in DMI for the given ranges

df_sponsor_temp = imsi_range.check_sponsor()

,IMSI_RANGE,S_IMSI_RANGE,PROFILE_NAME,NUM
0,260060149910,260060149910,P4_FreeCell,1000
1,260060149911,260060149911,P4_FreeCell,1000
2,260060149912,260060149912,P4_FreeCell,1000
3,260060149913,260060149913,P4_FreeCell,1000
4,260060149914,260060149914,P4_FreeCell,1000
...,...,...,...,...
81,454030227314,454030227314,MB_FreeCell,1000
82,454030227315,454030227315,MB_FreeCell,1000
83,454030227316,454030227316,MB_FreeCell,1000
84,454030227317,454030227317,MB_FreeCell,1000


In [28]:
# Prepare TOC commands to change the Sponsor in DMI

imsi_range.prepare_toc_to_cange_sponsor(downloads, out_file)

,NAME,IMSI
0,MB_ALL,11000
1,P4_All,36013
2,Partner_All,36012


,NAME,IMSI
0,MB_FreeCell,11000
1,P4_FreeCell,36013
2,Partner_FreeCell,36012


,CLI_MSISDN,IMSI,PROF_ID,IMSI_NUMBER,FLAG_CONFIRMED,NAME
83022,8529750012997,454030227318997,1,454030227318997, ,MB_FreeCell
83023,8529750012998,454030227318998,1,454030227318998, ,MB_FreeCell
83024,8529750012999,454030227318999,1,454030227318999, ,MB_FreeCell


### Developments

In [80]:
reseller_name = 'Tottolli-Tech-Mobil'
presision = 12 # Digits in IMSI

sql_srt = """
SELECT RESELLER_NAME,SUBSTRING(IMSI,1,{1}) AS IMSI_RANGE,COUNT(*) AS NUM
,MIN(IMSI) AS MIN_IMSI,MAX(IMSI) AS MAX_IMSI
FROM
(
SELECT
r.RESELLER_NAME
,ss.STATUS AS SUB_STATUS
,si.IMSI
FROM RESELLERS r
INNER JOIN ACCOUNTS a
ON r.RESELLER_ID = a.RESELLER_ID
INNER JOIN SUBSCRIBERS sr
ON sr.ACCOUNT_ID = a.ACCOUNT_ID
INNER JOIN SUBSCRIBER_STATUS ss
ON ss.SUBSCRIBER_ID = sr.SUBSCRIBER_ID
INNER JOIN SUBSCRIBER_IMSIS si
ON ss.SUBSCRIBER_ID = si.SUBSCRIBER_ID
WHERE
r.deleted is NULL
AND ss.END_DATE IS NULL
--AND ss.status = 'Active'
AND r.RESELLER_NAME = '{0}'
) t
GROUP BY RESELLER_NAME,SUBSTRING(IMSI,1,{1})
ORDER BY SUBSTRING(IMSI,1,{1})
""".format(reseller_name, presision)
with rb.MssqlConnect('172.18.11.82', '10028', 'BSS', 'iKQVm40AZAmyRaw72LeY') as cnxn:
    df_ocs_rslr_rng = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
df_ocs_rslr_rng

,RESELLER_NAME,IMSI_RANGE,NUM,MIN_IMSI,MAX_IMSI
0,Tottolli-Tech-Mobil,234500026400,20,234500026400002,234500026400021
1,Tottolli-Tech-Mobil,234500026425,1000,234500026425000,234500026425999
2,Tottolli-Tech-Mobil,234500026426,550,234500026426000,234500026426549
3,Tottolli-Tech-Mobil,234500026427,1000,234500026427000,234500026427999
4,Tottolli-Tech-Mobil,234500026428,1000,234500026428000,234500026428999
...,...,...,...,...,...
68,Tottolli-Tech-Mobil,515030191003,1000,515030191003000,515030191003999
69,Tottolli-Tech-Mobil,515030191004,1000,515030191004000,515030191004999
70,Tottolli-Tech-Mobil,515030191005,1000,515030191005000,515030191005999
71,Tottolli-Tech-Mobil,515030191006,1000,515030191006000,515030191006999


In [81]:
df_ocs_rslr_rng.loc[df_ocs_rslr_rng.IMSI_RANGE.str.startswith('42501')]

,RESELLER_NAME,IMSI_RANGE,NUM,MIN_IMSI,MAX_IMSI
34,Tottolli-Tech-Mobil,425019613791,500,425019613791500,425019613791999
35,Tottolli-Tech-Mobil,425019613792,1000,425019613792000,425019613792999
36,Tottolli-Tech-Mobil,425019613793,1000,425019613793000,425019613793999
37,Tottolli-Tech-Mobil,425019613794,1000,425019613794000,425019613794999
38,Tottolli-Tech-Mobil,425019613795,500,425019613795000,425019613795499
39,Tottolli-Tech-Mobil,425019613797,1000,425019613797000,425019613797999
40,Tottolli-Tech-Mobil,425019613958,1000,425019613958000,425019613958999
41,Tottolli-Tech-Mobil,425019613959,1000,425019613959000,425019613959999
42,Tottolli-Tech-Mobil,425019613972,1000,425019613972000,425019613972999
43,Tottolli-Tech-Mobil,425019613973,1000,425019613973000,425019613973999


In [83]:
for imsi in df_ocs_rslr_rng.loc[df_ocs_rslr_rng.IMSI_RANGE.str.startswith('42501'), ['MIN_IMSI','MAX_IMSI']].values:
    print(f'{imsi[0]}-{imsi[1]},')

425019613791500-425019613791999,
425019613792000-425019613792999,
425019613793000-425019613793999,
425019613794000-425019613794999,
425019613795000-425019613795499,
425019613797000-425019613797999,
425019613958000-425019613958999,
425019613959000-425019613959999,
425019613972000-425019613972999,
425019613973000-425019613973999,
425019613974000-425019613974999,
425019613975000-425019613975999,
425019613976000-425019613976999,
425019613977000-425019613977999,
425019613978000-425019613978999,
425019613979000-425019613979999,
425019613980000-425019613980999,
425019613998071-425019613998520,
425019614000012-425019614000013,
425019614001570-425019614001570,
425019629932000-425019629932999,
425019629933000-425019629933999,
425019629934000-425019629934999,
425019629935000-425019629935999,
425019629938000-425019629938999,
425019629939000-425019629939999,


In [18]:
df_temp.groupby(df_temp.IMSI_START.astype('str').str.slice(0,5))['NUM'].sum()

IMSI_START
23450    2502
26003    1700
26006    4839
42501    5516
45403    2505
51503    2508
Name: NUM, dtype: int64

In [96]:
imsi_range = '26006'

df_ocs_rslr_rng = df_temp.loc[df_temp.IMSI_START.astype('str').str.startswith(imsi_range)]

for imsi in df_ocs_rslr_rng.loc[:, ['IMSI_START','IMSI_END']].values:
    print(f'{imsi[0]}-{imsi[1]},')

260060140895371-260060140895371,
260060140895401-260060140895402,
260060140895553-260060140895555,
260060140895558-260060140895559,
260060140895561-260060140895562,
260060140895565-260060140895566,
260060140895596-260060140895596,
260060140895598-260060140895601,
260060140895603-260060140895604,
260060140895606-260060140895606,
260060140895707-260060140895707,
260060140895711-260060140895712,
260060140895735-260060140895737,
260060140895739-260060140895739,
260060140895742-260060140895743,
260060140895745-260060140895747,
260060140895749-260060140895750,
260060140895800-260060140895800,
260060140895803-260060140895804,
260060140895806-260060140895813,
260060140895816-260060140895822,
260060140896091-260060140896098,
260060140896100-260060140896104,
260060140896106-260060140896106,
260060140896108-260060140896112,
260060140896114-260060140896114,
260060140896153-260060140896153,
260060140896158-260060140896160,
260060140896162-260060140896162,
260060140896165-260060140896169,
2600601408